In [ ]:
# https://www.kaggle.com/dansbecker/transfer-learning

In [1]:
from tensorflow import keras
from keras.applications.resnet50 import ResNet50
from keras.models import Sequential
from keras.layers import Dense

resnet_weights_path = "./models/resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5"

base_model = ResNet50(weights=resnet_weights_path, pooling='avg', include_top=False)
# For retraining purposes we want pre-trained weights except its TOP layer. We use the pre-trained weights to initialize for training new layer using new training images.

new_model = Sequential()
new_model.add(base_model)
new_model.add(Dense(units=2, activation='softmax'))

# Freeze the base model - we do not want to modify the previously trained weightings
new_model.layers[0].trainable = False


Using TensorFlow backend.


In [2]:
new_model.compile(optimizer='sgd', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
from keras.applications.resnet50 import preprocess_input
from keras.preprocessing.image import ImageDataGenerator

image_size = 256
data_generator = ImageDataGenerator(preprocessing_function=preprocess_input)


train_generator = data_generator.flow_from_directory(
        './data/images/train/',
        target_size=(image_size, image_size),
        batch_size=32,
        class_mode='categorical')

validation_generator = data_generator.flow_from_directory(
        './data/images/validate/',
        target_size=(image_size, image_size),
        class_mode='categorical')

new_model.fit_generator(
        train_generator,
        steps_per_epoch=3,
        validation_data=validation_generator,
        validation_steps=1)

Found 179 images belonging to 2 classes.
Found 6 images belonging to 2 classes.
Epoch 1/1


In [ ]:
new_model.save("./models/retrained.h5")

In [ ]:
from keras.models import load_model
from keras.preprocessing import image
from keras.applications.resnet50 import preprocess_input, decode_predictions
import numpy as np

model = load_model('./models/retrained.h5')

img_path = 'figures/chili-dog-horiz-a-1600.jpg'
img = image.load_img(img_path, target_size=(256, 256))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = preprocess_input(x)

preds = model.predict(x)
# decode the results into a list of tuples (class, description, probability)
# (one such list for each sample in the batch)
#print('Predicted:', decode_predictions(preds))
print(preds)